In [1]:
import pathlib
import sys
import time

sys.path.append("../featurization_utils")
import gc

import mahotas
import numpy as np
import pandas as pd
import scipy
import skimage
import tqdm
from loading_classes import ImageSetLoader, ObjectLoader
from texture_utils import measure_3D_texture

try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False
if in_notebook:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

In [2]:
image_set_path = pathlib.Path("../../data/NF0014/cellprofiler/C4-2/")

In [3]:
channel_mapping = {
    "DNA": "405",
    "AGP": "488",
    "ER": "555",
    "Mito": "640",
    "BF": "TRANS",
    "Nuclei": "nuclei_",
    "Cell": "cell_",
    "Cytoplasm": "cytoplasm_",
    "Organoid": "organoid_",
}

In [4]:
image_set_loader = ImageSetLoader(
    image_set_path=image_set_path,
    anisotropy_spacing=(1, 0.1, 0.1),
    channel_mapping=channel_mapping,
)

In [5]:
start_time = time.time()

In [ ]:
for compartment in tqdm(
    image_set_loader.compartments, desc="Processing compartments", position=0
):
    for channel in tqdm(
        image_set_loader.image_names,
        desc="Processing channels",
        leave=False,
        position=1,
    ):
        object_loader = ObjectLoader(
            image_set_loader.image_set_dict[channel],
            image_set_loader.image_set_dict[compartment],
            channel,
            compartment,
        )
        output_texture_dict = measure_3D_texture(
            object_loader=object_loader,
            distance=1,
        )
        final_df = pd.DataFrame(output_texture_dict)

        final_df = final_df.pivot(
            index="object_id",
            columns="texture_name",
            values="texture_value",
        )
        final_df.reset_index(inplace=True)
        final_df.insert(0, "image_set", image_set_loader.image_set_name)
        final_df.columns.name = None

        output_file = pathlib.Path(
            f"../results/{image_set_loader.image_set_name}/Texture_{compartment}_{channel}_features.parquet"
        )
        output_file.parent.mkdir(parents=True, exist_ok=True)
        final_df.to_parquet(output_file)
        final_df.head()

56it [04:48,  5.15s/it]


,image_set,object_id,Texture_Angular.Second.Moment_256.1,Texture_Contrast_256.1,Texture_Correlation_256.1,Texture_Difference.Entropy_256.1,Texture_Difference.Variance_256.1,Texture_Entropy_256.1,Texture_Information.Measure.of.Correlation.1_256.1,Texture_Information.Measure.of.Correlation.2_256.1,Texture_Inverse.Difference.Moment_256.1,Texture_Sum.Average_256.1,Texture_Sum.Entropy_256.1,Texture_Sum.Variance_256.1,Texture_Variance_256.1
0,C4-2,1,0.999864,1.702440,0.300448,0.001382,0.003890,0.001552,-0.238117,0.014255,0.999935,0.015797,0.001458,4.511576,1.553504
1,C4-2,2,0.998032,2.856681,0.934248,0.016764,0.003884,0.023227,-0.596643,0.139675,0.999080,0.277402,0.019058,84.455286,21.827992
2,C4-2,3,0.999543,1.235051,0.826363,0.004473,0.003889,0.005707,-0.551098,0.065731,0.999784,0.050639,0.004936,13.600817,3.708967
3,C4-2,4,0.998376,1.941495,0.922132,0.014147,0.003885,0.019797,-0.590572,0.127946,0.999238,0.183453,0.016203,47.520696,12.365548
4,C4-2,5,0.998073,3.130576,0.900597,0.016670,0.003884,0.022973,-0.570814,0.134421,0.999093,0.227980,0.018812,59.363792,15.623592


In [ ]:
print(f"Time elapsed: {time.time() - start_time}")

Time elapsed: 289.5707528591156


: 